# Creating a bioMod file
This file shows how to create a `.bioMod` file, what should be declared, what it means, etc.
Additionnally, we explore how to generate the file programmatically. 
Please note that nothing prevents you from writting the file by hand.

In a nutshell a `.bioMod` file is a description of a rigid body kinematic chain. 
It therefore expect a serie of segments linked by degree of freedom, up to six (up to three translations and three rotations). 
Once loaded, if a degree of freedom is given to the model, then one can interact with it on this degree of freedom.

If one is interested only by the kinematics, then the inertial parameters can be omitted. 
Otherwise, they have to be declared.

The first section of this replicates the model that can be found in `models/SimpleBody.bioMod`

## Make our life easier
First of all, let's make our life easier by defining a class that we will fill for each of the segments and markers.

This section helps automatically write the file. 
If you want to write it by hand, you can skip this section.

In [ ]:
# First of all, let's make our life easier by defining a class that we will fill for each of the segment
class Segment():
    def __init__(
        self, 
        name, 
        parent_name="", 
        rt="",
        translations="", 
        rotations="", 
        mesh: tuple=None,
        markers: tuple=None,
    ):
        self.name = name
        self.parent_name = parent_name
        self.rt = rt
        self.translations = translations
        self.rotations = rotations
        self.mesh = mesh
        self.markers = markers

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"segment {self.name}\n"
        if (self.parent_name):
            out_string += f"\tparent {self.parent_name}\n"
        if (self.rt):
            out_string += f"\tRT {self.rt}\n"
        if (self.translations):
            out_string += f"\ttranslations {self.translations}\n"
        if (self.rotations):
            out_string += f"\trotations {self.rotations}\n"
        if (self.mesh):
            for m in self.mesh:
                out_string += f"\tmesh {m[0]} {m[1]} {m[2]}\n"
        out_string += "endsegment\n"
        
        # Also print the markers attached to the segment
        if (self.markers):
            for marker in self.markers:
                out_string += str(marker)
        return out_string
        
# Here is an example on how to use this class
my_new_segment = Segment(
    name="my_new_semgent_name", 
    parent_name="the_name_of_the_parent_segment", 
    rt="0 0 0 xyz 0 0 0",  # The transformation between the segment and its parent
    translations="yxz",  # The degrees of fredom in translation
    rotations="xyz",  # The degrees of fredom in rotation
    mesh=[[0, 0, 0], [0, 0, 0.3]]  # The list of mesh point to show
)
print(my_new_segment)  # Later this print should be sent to a file

In [ ]:
# Let's do the same for the markers
class Marker():
    def __init__(
        self, 
        name, 
        parent_name, 
        position,
    ):
        self.name = name
        self.parent_name = parent_name
        self.position = position

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"marker {self.name}\n"
        out_string += f"\tparent {self.parent_name}\n"
        out_string += f"\tposition {self.position[0]} {self.position[1]} {self.position[2]}\n"
        out_string += "endmarker\n"
        return out_string

# Here is an example on how to use this class
my_new_marker = Marker(
    name="my_new_marker_name",
    parent_name="the_name_of_the_parent_segment",
    position=[0, 0, 1],
)
print(my_new_marker)  # Later this print should be sent to a file

In [ ]:
# And why not creating a data structure that will collect all the segments and produce the a full
# kinematic chain that can easily be printed
class KinematicChain():
    def __init__(self, segments):
        self.segments = segments
        
    def __str__(self):
        out_string = "version 4\n\n"
        for segment in self.segments:
            out_string += str(segment)
            out_string += "\n\n\n"  # Give some space between segments
        return out_string
    
    def write(self, file_path):
        # Method to write the current KinematicChain to a file
        with open(file_path, "w") as file:
            file.write(str(self))

## Create the `SimpleBody.bioMod` kinematic chain
Now we are ready to produce the kinematic chain.
It is a 2D model made from:
- A TRUNK segment that includes the HEAD (up to the ear) at $72cm$ that moves frontward and upward (translations y and z) and rotations about the medio-lateral axis (rotations x);
- An ARM segment attached to the TRUNK at $53cm$ that includes the HAND at $61cm$. It rotates about the medio-lateral axis (rotations x);
- An THIGH segment attached to the TRUNK. It rotates about the medio-lateral axis (rotations x)
- An SHANK segment attached to the THIGH at $42cm$. It rotates about the medio-lateral axis (rotations x)
- An FOOT segment attached to the SHANK at $43cm$ that includes a FOOT of $18cm$. It rotates about the medio-lateral axis (rotations x)

In [ ]:
# Let's define the segments and the skin markers attached to each segment 
# and collect all the values in the KinematicChain class

# The trunk segment
trunk_marker_pelvis = Marker(
    name="PELVIS", 
    parent_name="TRUNK",
    position=[0, 0, 0],
)
trunk_marker_shoulder = Marker(
    name="SHOULDER", 
    parent_name="TRUNK",
    position=[0, 0, 0.53],
)
trunk_marker_head = Marker(
    name="EAR_HEAD", 
    parent_name="TRUNK",
    position=[0, 0, 0.72],
)
trunk = Segment(
    name="TRUNK",
    translations="yz", 
    rotations="x", 
    mesh=[[0, 0, 0], [0, 0, 0.72]],
    markers=[trunk_marker_pelvis, trunk_marker_shoulder, trunk_marker_head]
)



# The arm segment
arm_marker_hand = Marker(
    name="HAND", 
    parent_name="ARM",
    position=[0, 0, -0.61],
)
arm = Segment(
    name="ARM",
    parent_name=trunk.name, 
    rt="0 0 0 xyz 0 0 0.53", 
    rotations="x", 
    mesh=[[0, 0, 0], [0, 0, -0.61]],
    markers=[arm_marker_hand],
)



# The thigh segment
thigh_marker_knee = Marker(
    name="KNEE", 
    parent_name="THIGH",
    position=[0, 0, -0.42],
)
thigh = Segment(
    name="THIGH", 
    parent_name=trunk.name, 
    rotations="x", 
    mesh=[[0, 0, 0], [0, 0, -0.42]],
    markers=[thigh_marker_knee],
)


# The shank segment
shank_marker_knee = Marker(
    name="ANKLE", 
    parent_name="SHANK",
    position=[0, 0, -0.43],
)
shank = Segment(
    name="SHANK", 
    parent_name=thigh.name, 
    rt="0 0 0 xyz 0 0 -0.42", 
    rotations="x", 
    mesh=[[0, 0, 0], [0, 0, -0.43]],
    markers=[shank_marker_knee],
)


# The foot segment
foot_marker_toe = Marker(
    name="TOE", 
    parent_name="FOOT",
    position=[0, 0.18, 0],
)
foot = Segment(
    name="FOOT", 
    parent_name=shank.name, 
    rt="0 0 0 xyz 0 0 -0.43", 
    rotations="x", 
    mesh=[[0, 0, 0], [0, 0.18, 0]],
    markers=[foot_marker_toe],
)

kinematic_chain = KinematicChain(segments=[trunk, arm, thigh, shank, foot])
print(kinematic_chain)

In [ ]:
# Now let's generate the bioMod file. The file will be the same as the output of the previous cell
file_path = "models/my_new_model.bioMod"
kinematic_chain.write(file_path)

In [ ]:
# Run this cell to show the model you just created. 
# Please note that due to some jupyter limitations, you may have to restart after running this cell
# Please also note that this cell will crash if runned from the binder

import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(file_path)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.set_q([0, 0, -0.15, 1.20, 0.7, -1, 0.45])

# Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
viz.exec()

# If nothing happens, check for background. Sometimes the window loads behind the current window